# Lab 1: Métodos de Regresión

## ¡Bienvenido/a!

Te invitamos a realizar el primer trabajo.
- Objetivo: implementar modelos de regresión lineal y logística en Python.
- Tipo de actividad: Individual
- Tipo de evaluación: Sumativa 
- Ponderación: 12%
- Puntaje: 100 puntos
- Calificación: Escala de 1 a 7, con una exigencia de 50%. La nota mínima para aprobar es 4.0.

### Introducción

Esta tarea tiene como propósito abordar y profundizar en los aspectos del modelo de regresión lineal y logística. Se enfoca en las técnicas de selección de variables que aplican a estos modelos y se dan los primeros pasos para comprender cómo este tipo de modelos nos permite predecir o clasificar nuevas observaciones, cuantificando de alguna manera el error o la calidad del ajuste.


In [1]:
# Packages y módulos que utilizaremos 
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Bases de datos del Lab
import faraway.datasets.ozone as ozone
import faraway.datasets.prostate as prostate 

## Problema 1

Responda a las siguientes preguntas utilizando el conjunto de datos `ozone`. Considere $O^3$ como variable respuesta, mientras que las demás variables serán consideradas como variables explicativas

In [2]:
data = ozone.load()  # El objeto data contiene los datos para el problema 1

#### 1.1 - Realizar una descomposición aleatoria de la base de datos con la proporción 80%-20% para train y test, respectivamente. Utilice el número 123 como semilla de aleatorización.

In [3]:
## Guardar la información en:
# --> train: partición de entrenamiento    (incluyendo todas las variables)
# --> test: partición para la validacíon   (incluyendo todas las variables)

## Aquí va su código 

train, test = None, None    # Modifique estos objetos

# your code here
train, test = train_test_split(data, test_size=0.2, random_state=123)

In [4]:
# Testing para la pregunta 1.1

#### 1.2- Utilizando el conjunto de entrenamiento, ajuste un modelo de regresión lineal múltiple por mínimos cuadrados ordinarios. Considere el modelo con todas las variables disponibles (incluyendo intercepto). Determine que variables son estadísticamente significativas al 5%. 

In [5]:
# Debe entregar un vector booleano (True/False) que indique si la variable es significativa (True) o no (False).
# El orden de las variables en el vector debe ser: [const, vh, wind, humidity, temp, ibh, dpg, ibt, vis, doy]
# const = intercepto. 
# El vector debe tener por nombre sigVars
# Ejemplo: sigVars = [True, False, True, False, True, False, True, False, True, False]

## Aquí va su código 

sigVars = None # Modifique este objeto 

# your code here
X_train = train.drop(columns=["O3"])
X_train = sm.add_constant(X_train)
y_train = train["O3"]
X_test = test.drop(columns=["O3"])
X_test = sm.add_constant(X_test)
y_test = test["O3"]
model_lr_ols = sm.OLS(y_train, X_train).fit()
sigVars = model_lr_ols.pvalues < 0.05
sigVars

const       False
vh          False
wind        False
humidity     True
temp         True
ibh         False
dpg         False
ibt          True
vis         False
doy          True
dtype: bool

In [6]:
# Testing 1 para la pregunta 1.2

In [7]:
# Testing 2 para la pregunta 1.2

#### 1.3 - ¿Cuál es el coeficiente de determinación del modelo ajustado anteriormente?

In [8]:
# Guarde el coeficiente de determinación ($R^2$) en un objeto númerico de nombre R2.
# En caso de redondear, considerar al menos 4 decimales de precisión

R2 = None # Modifique este objeto 

# your code here
R2 = model_lr_ols.rsquared
R2 = round(R2, 4)
R2

0.6938

In [9]:
# Testing para la pregunta 1.3

#### 1.4 - ¿Cuál es el error cuadrático medio del modelo ajustado anteriormente? 

Recuerde que el MSE se define como:

\begin{equation*}
 MSE=\dfrac{1}{n}\sum_{i=1}^{n}e_i^2
\end{equation*}

In [10]:
# Guardar el error cuadrático medio en un objeto númerico de nombre MSE
MSE = None # Modifique este objeto 

# your code here
y_pred = model_lr_ols.predict(X_train)
MSE = metrics.mean_squared_error(y_train, y_pred)
MSE


19.550811615813686

In [11]:
# Testing para la pregunta 1.4

#### 1.5 - Construya un segundo modelo de regresión lineal múltiple (sin intercepto) utilizando las variables significativas del punto anterior. Luego, compare ambos modelos según AIC (Akaike Information Criterion). Según esta métrica, ¿cuál modelo es preferible?. 

Entregar el AIC de cada modelo en un vector de dos dimensiones de nombre `AIC`. Donde el primer elemento corresponde al AIC del modelo con todas las variables y el último elemento el AIC del modelo que solo incluye las variables significativas. Además, entregar un booleano de nombre `preferible` que indique si el modelo con menos variables es preferible (True) o no (False).

In [12]:
# your code here
X2_train = train.drop(columns=["O3"])
y2_train = train["O3"]
X2_test = test.drop(columns=["O3"])
y2_test = test["O3"]
X2_train = X2_train[['humidity', 'temp', 'ibt', 'doy']]
X2_test = X2_test[['humidity', 'temp', 'ibt', 'doy']]

model_lr_ols2 = sm.OLS(y2_train, X2_train).fit()

AIC = []
AIC.append(model_lr_ols.aic)
AIC.append(model_lr_ols2.aic)

if AIC[0] < AIC[1]:
    print("Modelo 1 has a lower AIC")
    preferible = False
else:
    print("Modelo 2 has a lower AIC")
    preferible = True

print(AIC)
print(preferible)

Modelo 1 has a lower AIC
[1554.0759808495568, 1597.020739159959]
False


In [13]:
# Testing para la pregunta 1.5

#### 1.6 - Ajuste un tercer modelo de regresión lineal (sin intercepto), pero ahora utilizando la técnica de selección de variables "forward stepwise", basada en el AIC. Para este caso, debe comenzar con el modelo que solo incluye las variables significativas del punto 3. Luego, debe agregar una variable a la vez, seleccionando la variable que minimice el AIC. Este proceso debe repetirse hasta que no sea posible agregar más variables. ¿Cuál es el AIC de este modelo? ¿Cuáles variables fueron seleccionadas?¿Es estadísticamente significativo el modelo? (Test F).

Entregar el AIC del modelo en un vector de una dimensión de nombre `AIC_forward`. Además, entregar un vector con el nombre de las `variables_seleccionadas`. Utilizar los nombres: vh, wind, humidity, temp, ibh, dpg, ibt, vis y doy. Por ejemplo, si wind y humidity son variables seleccionadas en este modelo, entonces 
`variables_seleccionadas = ['wind','humidity']`.

También, debe responder si este modelo es preferible a los anteriores según AIC. Entregar un booleano de nombre `preferible_forward` que indique si el modelo con menos variables es preferible (True) o no (False). 

Finalmente guardar el estadístico de prueba y el p-valor de la prueba F en un objeto denominado `testF` el cual será una lista donde `testF[0]`debe contener el estadístico de prueba, mientras que `testF[1]` debe contener el p-valor. Por ejemplo, si `model` es el modelo final seleccionado entonces `testF = [model.fvalue, model.f_pvalue]`.

In [34]:
AIC_forward = None                # Variable que debe modificar
variables_seleccionadas = None    # Variable que debe modificar
preferible_forward = None         # Variable que debe modificar

all_columns = ['vh','wind','humidity','temp','ibh','dpg','ibt','vis','doy']
selected_model2_columns = ['temp', 'ibt', 'doy']
rest_columns = list(set(all_columns) - set(selected_model2_columns))

X3_train = train[selected_model2_columns].copy()
y3_train = train["O3"].copy()
X_full = train[all_columns].copy()

model_lr_ols3 = sm.OLS(y3_train, X3_train).fit()
aic_curr = model_lr_ols3.aic
selected = selected_model2_columns.copy()

improved = True
while improved and len(rest_columns) > 0:
    improved = False
    best_var = None
    best_aic = aic_curr
    best_model = model_lr_ols3

    for var in rest_columns:
        cols_try = selected + [var]
        model_try = sm.OLS(y3_train, X_full[cols_try]).fit()
        if model_try.aic < best_aic:
            best_aic = model_try.aic
            best_var = var
            best_model = model_try

    if best_var is not None:
        selected.append(best_var)
        rest_columns.remove(best_var)
        model_lr_ols3 = best_model
        aic_curr = best_aic
        improved = True

AIC_forward = float(aic_curr)

variables_seleccionadas = selected
testF = [model_lr_ols3.fvalue, model_lr_ols3.f_pvalue]
preferible_forward = True
print("preferible_forward:", preferible_forward)
print("AIC_forward:", AIC_forward)
print("variables_seleccionadas:", variables_seleccionadas)
print(f"Test F -> F={testF[0]:.4f}, p-value={testF[1]:.4g}")

preferible_forward: True
AIC_forward: 1547.6899975652686
variables_seleccionadas: ['temp', 'ibt', 'doy', 'vh', 'humidity', 'vis']
Test F -> F=410.7837, p-value=6.735e-129


In [33]:
# Testing 1 para la pregunta 1.6

In [16]:
# Testing 2 para la pregunta 1.6

#### 1.7 - Con base al modelo del punto anterior, realice la predicción de la variable respuesta para el conjunto de test. Luego, calcule el error cuadrático medio de la predicción. 

Entregar el error cuadrático medio en una variable de nombre `MSEpred` y el vector de predicciones en una variable de nombre `pred`.

In [17]:
pred = None         # Variable que debe modificar
MSEpred = None      # Variable que debe modificar

# your code here
X3_test = test[variables_seleccionadas].copy()
y3_test = test["O3"].copy()
pred = model_lr_ols3.predict(X3_test)
MSEpred = float(np.mean((y3_test - pred) ** 2))
MSEpred

17.65874898760734

In [18]:
# Testing para la pregunta 1.7

# Problema 2

Para esta segunda sección, se utilizará el conjunto de datos `prostate`. Este conjunto de datos contiene información sobre 97 pacientes con cáncer de próstata. Considere la variable respuesta como `svi` (seminal vesicle invasion). Esta variable indica si el cáncer se ha extendido a la vesícula seminal (1) o no (0). Las demás variables serán consideradas como variables explicativas. El 
objetivo es ajustar un modelo de regresión logística para predecir la invasión de la vesícula seminal. Los datos se cargarán automáticamente en la siguiente celda, con el nombre `prostate_data`.

In [19]:
prostate_data = prostate.load()  # cargamos los datos del problema 2

#### 2.1 - Realizar una descomposición aleatoria de la base de datos con la proporción 80%-20% para train y test, respectivamente. Utilice el número 123 como semilla de aleatorización.

In [20]:
## Guardar la información en:
# --> train_prostate: partición de entrenamiento    (incluyendo todas las variables)
# --> test_prostate: partición para la validacíon   (incluyendo todas las variables)

## Aquí va su código 

train_prostate, test_prostate = None, None    # Modifique estos objetos

# your code here
train_prostate, test_prostate = train_test_split(prostate_data, test_size=0.2, random_state=123)

In [21]:
# Testing para la pregunta 2.1

#### 2.2 - Ajuste un modelo de regresión logística usando todas las variables explicativas (sin intercepto). Luego, realice las predicciones para el conjunto de datos de test. Finalmente, calcule el porcentaje de clasificación correcta.

Entregar el porcentaje de clasificación correcta en una variable de nombre `porcentaje_clasificacion`. 

In [22]:
# Aquí va su código

porcentaje_clasificacion = None # Variable que debe modificar

# your code here
X_train = train_prostate.drop(columns="svi", axis=1)
y_train = train_prostate["svi"]
X_test = test_prostate.drop(columns="svi", axis=1)
y_test = test_prostate["svi"]

model_logit = sm.Logit(y_train, X_train).fit()

y_pred_prob = model_logit.predict(X_test)
y_pred_class = (y_pred_prob >= 0.5).astype(int)
accuracy = metrics.accuracy_score(y_test, y_pred_class)
porcentaje_clasificacion = accuracy
porcentaje_clasificacion

Optimization terminated successfully.
         Current function value: 0.162442
         Iterations 9


0.7

In [23]:
# Testing para la pregunta 2.2

#### 2.3 - ¿El modelo anterior presenta problemas de multicolinealidad? Para responder esta pregunta, calcule el factor de inflación de la varianza (VIF) para cada variable. ¿Cuáles variables presentan mayor VIF? 

Para responder esta pregunta, entregar un vector de nombre `VIF_vars` que contenga el VIF de cada variable. El orden de las variables en el vector debe ser: lcavol, lweight, age, lbph, lcp, gleason, pgg45, lpsa. 

Recordar que:
- VIF = 1: Las variables no están correlacionadas.
- 1 < VIF < 5: Las variables están moderadamente correlacionadas y pueden ser incluidas en el modelo.
- VIF > 5: Las variables están altamente correlacionadas y deben ser eliminadas del modelo. 

TIP: puede utilizar la función `variance_inflation_factor` del paquete `statsmodels` para calcular el VIF.

In [28]:
# Aquí va su código

VIF_vars = None # Variable que debe modificar

# your code here
vars_order = ['lcavol', 'lweight', 'age', 'lbph', 'lcp', 'gleason', 'pgg45', 'lpsa']

X_vif = X_train[vars_order].copy()
VIF_vars = [variance_inflation_factor(X_vif.values, i) for i in range(X_vif.shape[1])]

VIF_vars = [float(v) for v in VIF_vars] 
VIF_vars

[6.749196983258511,
 57.78393933139402,
 82.40542217672011,
 1.195335723452414,
 2.531763153388447,
 84.90034919124903,
 3.809147884382757,
 16.916887569167937]

In [29]:
# Testing para la pregunta 2.3

#### 2.4 Elimine todas las variables que presenten VIF > 10. Luego, realice un proceso de selección de variables utilizando la técnica "backward stepwise", basada en el valor-p. Para este caso, debe comenzar con el modelo (con intercepto) que incluye todas las variables con VIF < 10. Luego, debe eliminar una variable a la vez, seleccionando la variable menos significativa (mayor valor-p). Este proceso debe repetirse hasta que no sea posible eliminar más variables, es decir, hasta que todas las variables sean estadísticamente significativas (valor-p < 0.05). ¿Cuál es el porcentaje de clasificación correcta de este nuevo modelo?

Debe entregar el porcentaje de clasificación correcta en una variable de nombre `porcentaje_clasificacion_backward`.

In [30]:
# Aquí va su código

porcentaje_clasificacion_backward = None # Variable que debe modificar

# your code here
df_vif = pd.DataFrame({
    "Features": vars_order,
    "VIF": VIF_vars
})

df_vif_filtered = df_vif[df_vif["VIF"] < 10]
selected_feats = df_vif_filtered['Features'].tolist()

while True:
    Xb_train = sm.add_constant(train_prostate[selected_feats], has_constant="add")
    logit_fit = sm.Logit(y_train, Xb_train).fit(disp=0)

    pvals = logit_fit.pvalues.drop(labels=["const"], errors="ignore")

    if pvals.empty or (pvals.max() < 0.05):
        break

    worst_feat = pvals.idxmax()
    selected_feats.remove(worst_feat)

Xb_test = sm.add_constant(test_prostate[selected_feats], has_constant="add")
y_pred_prob_b = logit_fit.predict(Xb_test)
y_pred_class_b = (y_pred_prob_b >= 0.5).astype(int)
accuracy_b = metrics.accuracy_score(y_test, y_pred_class_b)
porcentaje_clasificacion_backward = accuracy_b
porcentaje_clasificacion_backward

0.75

In [31]:
# Testing para la pregunta 2.4